In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['database.csv']


In [2]:
import xgboost as xgb
import matplotlib.pyplot as plt
import math as mth

In [3]:
earthquakes = pd.read_csv("./input/database.csv")

In [4]:
earthquakes.describe()

,Latitude,Longitude,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Error,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square
count,23412.000000,23412.000000,23412.000000,4461.000000,7097.000000,23412.000000,327.000000,2564.000000,7299.000000,1604.000000,1156.000000,17352.000000
mean,1.679033,39.639961,70.767911,4.993115,275.364098,5.882531,0.071820,48.944618,44.163532,3.992660,7.662759,1.022784
std,30.113183,125.511959,122.651898,4.875184,162.141631,0.423066,0.051466,62.943106,32.141486,5.377262,10.430396,0.188545
min,-77.080000,-179.997000,-1.100000,0.000000,0.000000,5.500000,0.000000,0.000000,0.000000,0.004505,0.085000,0.000000
25%,-18.653000,-76.349750,14.522500,1.800000,146.000000,5.600000,0.046000,10.000000,24.100000,0.968750,5.300000,0.900000
50%,-3.568500,103.982000,33.000000,3.500000,255.000000,5.700000,0.059000,28.000000,36.000000,2.319500,6.700000,1.000000
75%,26.190750,145.026250,54.000000,6.300000,384.000000,6.000000,0.075500,66.000000,54.000000,4.724500,8.100000,1.130000
max,86.005000,179.998000,700.000000,91.295000,934.000000,9.100000,0.410000,821.000000,360.000000,37.874000,99.000000,3.440000


In [5]:
earthquakes.head()

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic


In [6]:
earthquakes[['Magnitude', 'Magnitude Type']].groupby(['Magnitude Type'], as_index=False).mean().sort_values(by='Magnitude', ascending=False)

,Magnitude Type,Magnitude
2,MH,6.540000
9,MWW,6.008674
4,MS,5.994360
1,MD,5.966667
5,MW,5.933794
6,MWB,5.907282
7,MWC,5.858176
3,ML,5.814675
0,MB,5.682957
8,MWR,5.630769


In [7]:
earthquakes[['Magnitude', 'Type']].groupby(['Type'], as_index=False).mean().sort_values(by='Magnitude', ascending=False)

,Type,Magnitude
3,Rock Burst,6.200000
0,Earthquake,5.882763
2,Nuclear Explosion,5.850686
1,Explosion,5.850000


In [8]:
for k in earthquakes:
    k_vals = earthquakes[k]
    print(k,k_vals.dropna().shape)

Date (23412,)
Time (23412,)
Latitude (23412,)
Longitude (23412,)
Type (23412,)
Depth (23412,)
Depth Error (4461,)
Depth Seismic Stations (7097,)
Magnitude (23412,)
Magnitude Type (23409,)
Magnitude Error (327,)
Magnitude Seismic Stations (2564,)
Azimuthal Gap (7299,)
Horizontal Distance (1604,)
Horizontal Error (1156,)
Root Mean Square (17352,)
ID (23412,)
Source (23412,)
Location Source (23412,)
Magnitude Source (23412,)
Status (23412,)


In [9]:
earthquakes = earthquakes.drop(["Depth Error", "Depth Seismic Stations", "Magnitude Error", 
                  "Magnitude Seismic Stations", "Azimuthal Gap", "Horizontal Distance", "Horizontal Error",
                  "ID", "Source", "Location Source", "Magnitude Source", "Status"], axis=1)

In [12]:
earthquakes.head()

,Date,Time,Latitude,Longitude,Type,Depth,Magnitude,Magnitude Type,Root Mean Square
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,6.0,MW,NaN
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,5.8,MW,NaN
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,MW,NaN
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,MW,NaN
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,5.8,MW,NaN


In [11]:
dt = pd.to_datetime(earthquakes["Date"], infer_datetime_format=True)

In [13]:
earthquakes['Y'] = dt.dt.year
earthquakes['M'] = dt.dt.month

In [14]:
earthquakes.head()

,Date,Time,Latitude,Longitude,Type,Depth,Magnitude,Magnitude Type,Root Mean Square,Y,M
0,01/02/1965,13:44:18,19.246,145.616,Earthquake,131.6,6.0,MW,NaN,1965,1
1,01/04/1965,11:29:49,1.863,127.352,Earthquake,80.0,5.8,MW,NaN,1965,1
2,01/05/1965,18:05:58,-20.579,-173.972,Earthquake,20.0,6.2,MW,NaN,1965,1
3,01/08/1965,18:49:43,-59.076,-23.557,Earthquake,15.0,5.8,MW,NaN,1965,1
4,01/09/1965,13:32:50,11.938,126.427,Earthquake,15.0,5.8,MW,NaN,1965,1


In [15]:
eq = earthquakes.drop(['Date', 'Time'], axis=1)

In [16]:
eq['sin_lat'] = np.sin(eq['Latitude'])
eq['cos_lat'] = np.cos(eq['Latitude'])
eq['cos_long'] = np.cos(eq['Longitude'])
eq['sin_long'] = np.sin(eq["Longitude"])

In [17]:
eq = eq.drop(['Latitude', 'Longitude'], axis=1)

In [18]:
eq.head()

,Type,Depth,Magnitude,Magnitude Type,Root Mean Square,Y,M,sin_lat,cos_lat,cos_long,sin_long
0,Earthquake,131.6,6.0,MW,NaN,1965,1,0.386141,0.922440,0.451154,0.892446
1,Earthquake,80.0,5.8,MW,NaN,1965,1,0.957611,-0.288063,-0.117227,0.993105
2,Earthquake,20.0,6.2,MW,NaN,1965,1,-0.987442,-0.157983,-0.376849,0.926275
3,Earthquake,15.0,5.8,MW,NaN,1965,1,-0.576354,-0.817200,-0.004945,0.999988
4,Earthquake,15.0,5.8,MW,NaN,1965,1,-0.587827,0.808986,0.722563,0.691305


In [19]:
labels = eq['Magnitude']
eq.drop('Magnitude', axis=1, inplace=True)

In [20]:
eq.insert(eq.shape[1], 'Magnitude', labels)

In [21]:
mag_types = eq['Magnitude Type'].unique()
mag_types

array(['MW', 'ML', 'MH', 'MS', 'MB', 'MWC', 'MD', nan, 'MWB', 'MWW',
       'MWR'], dtype=object)

In [22]:
replacement = {v:k for k,v in enumerate(mag_types) if type(v) is str}
replacement

{'MW': 0,
 'ML': 1,
 'MH': 2,
 'MS': 3,
 'MB': 4,
 'MWC': 5,
 'MD': 6,
 'MWB': 8,
 'MWW': 9,
 'MWR': 10}

In [23]:
eq = eq.replace(replacement)

In [24]:
types = eq['Type'].unique()
types

array(['Earthquake', 'Nuclear Explosion', 'Explosion', 'Rock Burst'],
      dtype=object)

In [25]:
replacement = {v:k for k,v in enumerate(types)}
replacement

{'Earthquake': 0, 'Nuclear Explosion': 1, 'Explosion': 2, 'Rock Burst': 3}

In [26]:
eq = eq.replace(replacement)

In [27]:
eq.head()

,Type,Depth,Magnitude Type,Root Mean Square,Y,M,sin_lat,cos_lat,cos_long,sin_long,Magnitude
0,0,131.6,0.0,NaN,1965,1,0.386141,0.922440,0.451154,0.892446,6.0
1,0,80.0,0.0,NaN,1965,1,0.957611,-0.288063,-0.117227,0.993105,5.8
2,0,20.0,0.0,NaN,1965,1,-0.987442,-0.157983,-0.376849,0.926275,6.2
3,0,15.0,0.0,NaN,1965,1,-0.576354,-0.817200,-0.004945,0.999988,5.8
4,0,15.0,0.0,NaN,1965,1,-0.587827,0.808986,0.722563,0.691305,5.8


In [28]:
eq.head()

,Type,Depth,Magnitude Type,Root Mean Square,Y,M,sin_lat,cos_lat,cos_long,sin_long,Magnitude
0,0,131.6,0.0,NaN,1965,1,0.386141,0.922440,0.451154,0.892446,6.0
1,0,80.0,0.0,NaN,1965,1,0.957611,-0.288063,-0.117227,0.993105,5.8
2,0,20.0,0.0,NaN,1965,1,-0.987442,-0.157983,-0.376849,0.926275,6.2
3,0,15.0,0.0,NaN,1965,1,-0.576354,-0.817200,-0.004945,0.999988,5.8
4,0,15.0,0.0,NaN,1965,1,-0.587827,0.808986,0.722563,0.691305,5.8


In [29]:
eq_dmatrix = xgb.DMatrix(data=eq.iloc[:,:-1], 
                         label=eq.iloc[:,-1])

In [30]:
params = {"objective":"reg:linear"}

In [76]:
xg_cl = xgb.XGBClassifier(objective='reg:linear', n_estimators=1, seed=123)

In [77]:
xg_cl.fit(eq.iloc[:15000,:-1], eq.iloc[:15000,-1])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=123,
       silent=True, subsample=1)

In [78]:
res = xg_cl.predict(eq.iloc[15000:,:-1])

C:\Users\KiriToK\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [79]:
res

array([5.5, 5.5, 5.5, ..., 5.7, 6. , 5.5])

In [80]:
eq.iloc[15000:,-1]

15000    5.8
15001    5.5
15002    5.8
15003    5.5
15004    5.7
15005    5.6
15006    5.9
15007    5.5
15008    5.8
15009    5.5
15010    6.1
15011    5.5
15012    5.5
15013    6.3
15014    6.1
15015    6.0
15016    6.2
15017    5.5
15018    5.5
15019    5.6
15020    5.9
15021    5.5
15022    5.6
15023    5.5
15024    6.2
15025    5.5
15026    5.7
15027    5.5
15028    5.9
15029    6.4
        ... 
23382    5.5
23383    6.0
23384    5.5
23385    5.5
23386    7.9
23387    5.6
23388    6.3
23389    5.9
23390    5.5
23391    5.9
23392    6.2
23393    5.5
23394    6.4
23395    6.4
23396    5.6
23397    6.0
23398    5.5
23399    6.7
23400    5.9
23401    6.0
23402    5.8
23403    7.6
23404    5.6
23405    5.6
23406    5.6
23407    5.6
23408    5.5
23409    5.9
23410    6.3
23411    5.5
Name: Magnitude, Length: 8412, dtype: float64

In [81]:
np.mean((res-eq.iloc[15000:,-1])**2)

0.26683392772229986